# Performance analysis of scientific code

## Bottlenecks in python code

### Background

It is often the case that we write code by directly transcribing scientific concepts (such as equations to solve numerical equations) into various programming idioms such as loops, by using the native data structures and built-in functionality of Python and its standard library.

However, surprisingly this straightforward approach sometimes yields in sub-optimal performance wherein it takes a significant amount of time to compute the results.

It is not that every operation in Python is slow. In fact, basic operations such as assigning values to variables, doing mathematical operations on scalars or on a small collection of entities, printing to the console, performing logical comparisons etc are usually fast enough to be not noticeable. In large scientific codebases, performance penalties typically arise within only a few sections of code that usually deal with certain critical operations involving large-scale numerical manipulations.

Hence, it is worthwhile to understand where the bottlenecks of the code lie, and how to mitigate them.

### Exercise: Exporing Python Performance with Computational Fluid Dynamics

#### Introduction to the exercise

This exercise takes an example from one of the most common applications of HPC resources: Fluid Dynamics. We will look at computational bottlenecks that arise in computing the results using naively written code.

#### Fluid Dynamics: a brief overview

Fluid Dynamics is the study of the mechanics of fluid flow, liquids and gases in motion. This can encompass aerodynamics and hydrodynamics. It has wide ranging applications from vessel and structure design to weather and traffic modelling. Simulating and solving fluid dynamic problems often requires large computational resources.

Fluid dynamics is an example of continuous system that can be described by Partial Differential Equations. For a computer to simulate these systems, the equations must be discretised onto a grid. If this grid is regular, then a finite difference approach can be used. Using this method means that the value at any point in the grid is updated using some combination of the neighbouring points.

_Discretisation_ is the process of approximating a continuous (i.e. infinite-dimensional) problem by a finite-dimensional problem suitable for a computer. This is often accomplished by putting the calculations into a grid or similar construct.

#### The Problem

In this exercise the finite difference approach is used to determine the flow pattern of a fluid in a cavity. For simplicity, the liquid is assumed to have zero viscosity, which implies that there can be no vortices (i.e. no whirlpools) in the flow. The cavity is a square box with an inlet on one side and an outlet on another as shown below.

![cavity image](./images/cavity.png)

#### Mathematical background (optional)

In two dimensions it is easiest to work with the stream function
$\psi$ (see below for how this relates to the fluid velocity). For zero viscosity, $\psi$ satisfies the following equation:

$$
\nabla^2 \psi = \frac{\partial^2 \psi}{\partial x^2}
$$

The finite difference version of this equation is:

$$
\psi_{i-1,j} + \psi_{i+1,j} + \psi_{i,j-1} + \psi_{i,j+1}
-4 \psi_{i,j} = 0.
$$

With the boundary values fixed, the stream function can be calculated for each point in the grid by averaging the value
at that point with its four nearest neighbours. The process continues until the algorithm converges on a solution that
stays unchanged by the averaging process. This simple approach
to solving a PDE is called the Jacobi algorithm.

In order to obtain the flow pattern of the fluid in the cavity
we want to compute the velocity field $\mathbf{u}(x,y)$. The $x$ and $y$ components of the velocity are related to the stream function by

$$
u_x =  \frac{\partial \psi}{\partial y} = \frac{1}{2}(\psi_{i,j+1} - \psi_{i,j-1}),
\quad
u_y = -\frac{\partial \psi}{\partial x} = \frac{1}{2}(\psi_{i+1,j}-\psi_{i-1,j}).
$$

This means that the velocity of the fluid at each grid point
can also be calculated from the surrounding grid points. The magnitude of the velocity $\mathbf{u}$ is
given by $u = (u_x^2 + u_y^2)^{1/2}$.

#### An algorithm

The outline of the algorithm for calculating the velocities is
as follows:

```
Set the boundary values for stream function
while (convergence is FALSE):
     for each interior grid point:
         update the stream function
     
     compute convergence criteria

for each interior grid point:
    compute x component of velocity
    compute y component of velocity
```

For simplicity, here we simply run the calculation for a fixed number of iterations; a real simulation would continue until some chosen accuracy was achieved.

#### Using Python

You are given a basic (but inefficient) starter code in `./cfd_python_lists` that uses Python lists to run the simulation. 
There are a number of different files:

```
 cfd_python_lists
 ├─ cfd.py         # python driver script
 └─ jacobi.py      # Jacobi algorthm code  
```
Look at the structure of the `cfd.py` code. In particular, note:

* How the external "jacobi" function is included
* How the lists are declared and initialised to zero
* How the timing works

#### Initial run

Jacobi iterations take long to converge, approximately at least 10000 steps are needed for an acceptable convergence this problem for a grid size of 128 x 128 (which is still not quite a realistic grid size). 

Navigate to the `cfd_python_lists` subdirectory and run the main program:
```bash
prompt:/path/to/cfd_python_lists> python cfd.py 4 10000
```

As the program is running you should see output that looks something like:

```
2D CFD Simulation
=================
Scale factor = 4
Iterations   = 10000

Initialisation took 0.00022s

Grid size = 128 x 128

Starting main Jacobi loop...
completed iteration 1000
completed iteration 2000
completed iteration 3000
completed iteration 4000
completed iteration 5000
completed iteration 6000
completed iteration 7000
completed iteration 8000
completed iteration 9000
completed iteration 10000

...finished

Calculation took 55.79600s
```

## Profiling the CFD example program

### Using `cProfile`

Python has a nice, built-in statistical profiling module called cProfile. You can use it to collect data from your program without having to manually add any instrumentation. Optionally, you can then visualize the data collected using additional tools such as `SnakeViz` and `gprof2dot`.

We will now profile the CFD program and collect data using `cprofile`:
```
python -m cProfile -o profile_data.prof ./cfd.py 4 10000
```

This example will generate a `profile_data.prof` file which contains the profiling data. Note that this is a binary (i.e. not plain-text) file which needs to be further processed by a suitable tool.

In [10]:
import pstats, os
stats = pstats.Stats(os.getcwd() + '/cfd_python_lists/profile_data.prof') # Please use the correct relative path to this file
stats.sort_stats('tottime')
stats.print_stats()

Mon Jul 29 15:17:19 2024    /home/krishnakumar/Documents/work_ucl_arc/arc_cluster_club/cluster_club_accelerated_python/cfd_python_lists/profile_data.prof

         7272 function calls (7095 primitive calls) in 53.100 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1   53.073   53.073   53.074   53.074 /home/krishnakumar/Documents/work_ucl_arc/arc_cluster_club/cluster_club_accelerated_python/cfd_python_lists/jacobi.py:10(jacobi)
       10    0.004    0.000    0.004    0.000 {built-in method _imp.create_dynamic}
       16    0.004    0.000    0.004    0.000 {built-in method marshal.loads}
    57/55    0.001    0.000    0.002    0.000 {built-in method builtins.__build_class__}
       81    0.001    0.000    0.004    0.000 <frozen importlib._bootstrap_external>:1593(find_spec)
        1    0.001    0.001   53.075   53.075 ./cfd.py:20(main)
      130    0.001    0.000    0.001    0.000 {built-in method posix.stat}
      

It is seen that the most time-consuming part of the code is the function calls spent in evaluating line 9 of `jacobi.py` which is:
```python
def jacobi(niter, psi):
```
This indicates to us that `jacobi` is an expensive function to evaluate. However, that is all we get to know at this stage. `cprofile` has a reasonably low overhead, but only gives information at the function call level, and not at the level of individual lines.

### Using a line profiler

Albeit not in the standard library, there exist third-party line profiling tools available as python libraries. We shall use the popular [`pyutils/line_profiler`](https://github.com/pyutils/line_profiler) line profiler (already listed in the `requirements.txt` for this workshop's environment) for further analysis of our CFD code.

#### Decorator syntax

The python interpreter can be given additional context and meaning on how to "process" _functions_ in a special manner. To achieve this, we use the decorator syntax, which is to prepend the line immediately above the function under consideration with a line of code starting with `@`. For the `line_profiler` library, the decorator syntax to be used is:

```python
@line_profiler.profile
def myfun(arg1,arg2,arg3):
   # some lines of function body
```

### Line profiler for CFD code
In our CFD example, we will first import the `line_profiler` at the top of the relevant file `jacobi.py`. Next, we decorate the `jacobi` function in `jacobi.py`. Not that `line_profiler` is quite invasive and has high overheads. Therefore, to find out the bottlenecks, it may be appropriate to reduce the grid size (e.g. use 64 x 64 grid i.e. a scale factor of 3) and reduce the number of jacobi iterations (say 5000). Finally, we set the environment variable LINE_PROFILE=1 as shown below, and run the script as normal:

>On Linux and macOS
>```bash
>prompt:/path/to/cfd_python_lists> LINE_PROFILE=1 python cfd.py 3 5000
>```


> On Windows
>```
>prompt:C:\path\to\cfd_python_lists> setx LINE_PROFILE 1
>prompt:C:\path\to\cfd_python_lists> python cfd.py 3 5000
>```

When the script finishes, a summary of profile results are made available with output files written to disk, and instructions for inspecting details shown on screen.

Typically, the line profiler will output 3 files: `profile_output.txt`, `profile_output_<timestamp>.txt`, and `profile_output.lprof` and the last few lines of console output will look something like:

```
Timer unit: 1e-09 s

 98.94 seconds - /home/krishnakumar/Documents/work_ucl_arc/arc_cluster_club/cluster_club_accelerated_python/cfd_python_lists/jacobi.py:10 - jacobi
Wrote profile results to profile_output.txt
Wrote profile results to profile_output_2024-07-19T133938.txt
Wrote profile results to profile_output.lprof
To view details run:
python -m line_profiler -rtmz profile_output.lprof
```

### Viewing the line profiler results

We follow the instructions in the last line of line profiler's output, and invoke:

```bash
python -m line_profiler -rtmz profile_output.lprof
```

which produces results alike:

```
Total time: 98.9386 s
File: cfd_python_lists/jacobi.py
Function: jacobi at line 10

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    10                                           @line_profiler.profile
    11                                           def jacobi(niter, psi):
    12
    13                                               # Get the inner dimensions
    14         1          0.8      0.8      0.0      m = len(psi) - 2
    15         1          0.3      0.3      0.0      n = len(psi[0]) -2
    16
    17                                               # Define the temporary list and zero it
    18     17031       2596.9      0.2      0.0      tmp = [[0 for col in range(n+2)] for row in range(m+2)]
    19
    20                                               # Iterate for number of iterations
    21     10000       3159.6      0.3      0.0      for iter in range(1,niter+1):
    22
    23                                                   # Loop over the elements computing the stream function
    24   1290000     170220.1      0.1      0.2          for i in range(1,m+1):
    25 165120000   23959696.1      0.1     24.2              for j in range(1,n+1):
    26 163840000   31154880.0      0.2     31.5                  tmp[i][j] = 0.25 * (psi[i+1][j]+psi[i-1][j]+psi[i][j+1]+psi[i][j-1])
    27
    28                                                   # Update psi
    29   1290000     178970.8      0.1      0.2          for i in range(1,m+1):
    30 165120000   23053866.7      0.1     23.3              for j in range(1,n+1):
    31 163840000   20408242.6      0.1     20.6                  psi[i][j] = tmp[i][j]
    32
    33                                                   # Debug output
    34     10000       6647.4      0.7      0.0          if iter%1000 == 0:
    35        10        321.9     32.2      0.0              sys.stdout.write("completed iteration {0}\n".format(iter))

 98.94 seconds - cfd_python_lists/jacobi.py:10 - jacobi
```

### Interpreting the line profiler results

The most salient lines in the line profile's results are the following:

```
Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    18     17031       2596.9      0.2      0.0      tmp = [[0 for col in range(n+2)] for row in range(m+2)]
    21     10000       3159.6      0.3      0.0      for iter in range(1,niter+1):
    23                                                   # Loop over the elements computing the stream function
    24   1290000     170220.1      0.1      0.2          for i in range(1,m+1):
    25 165120000   23959696.1      0.1     24.2              for j in range(1,n+1):
    26 163840000   31154880.0      0.2     31.5                  tmp[i][j] = 0.25 * (psi[i+1][j]+psi[i-1][j]+psi[i][j+1]+psi[i][j-1])
    29   1290000     178970.8      0.1      0.2          for i in range(1,m+1):
    30 165120000   23053866.7      0.1     23.3              for j in range(1,n+1):
    31 163840000   20408242.6      0.1     20.6                  psi[i][j] = tmp[i][j]


There are two key observations. The most time-consuming parts of the function are:
1. Looping through the elements of the 2D array leads to significant performance hits.
2. Constructing and zero initialising a python list of lists using list comprehension

## Next steps
In the subsequent sessions, we will see how to address these code bottlenecks to improve performance